必要ModuleをImport

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import math

import random
import time
import pickle
import tqdm
from collections import Counter

from torch.utils.data import Dataset
import numpy as np

from utils import GELU, PositionwiseFeedForward, LayerNorm, SublayerConnection, LayerNorm

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import FloatProgress
from IPython.display import display, clear_output

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

read_dir = './data/merged_bert/'
output_dir = './output/merged_bert/'
processed_train_txt = read_dir + 'train_X.txt'
processed_valid_txt = read_dir + 'valid_X.txt'
processed_test_txt = read_dir + 'test_X.txt'

Attentionセルを定義する

In [3]:
class Attention(nn.Module):
    """
    Scaled Dot Product Attention
    """
    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn


Multi Head Attentionを定義する

In [4]:
class MultiHeadedAttention(nn.Module):

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h

        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linear_layers, (query, key, value))]

        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x)


Transformerを定義する

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, hidden, attn_heads, feed_forward_hidden, dropout):
        super().__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model=hidden, d_ff=feed_forward_hidden, dropout=dropout)
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return self.dropout(x)


BERTクラスを定義する

In [6]:
class BERT(nn.Module):

    def __init__(self, vocab_size, hidden=768, n_layers=12, attn_heads=12, dropout=0.1):
        super().__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.attn_heads = attn_heads
        self.feed_forward_hidden = hidden * 4
        # embedding for BERT
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=hidden, dropout=dropout)

        self.transformer_blocks = nn.ModuleList([TransformerBlock(hidden, attn_heads, hidden * 4, dropout) for _ in range(n_layers)])

    def forward(self, x):
        # xの中で0以上は1, 0未満は0として, maskテンソルを作る
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)
        x = self.embedding(x)

        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)
        return x


BERTのEmbedding層を定義する

In [7]:
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)

class PositionalEmbedding(nn.Module):

    def __init__(self, d_model, max_len=512):
        super().__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)).float().exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class BERTEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size, dropout=0.1):
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(d_model=self.token.embedding_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    #def forward(self, sequence, segment_label):
    def forward(self, sequence):
        x = self.token(sequence) + self.position(sequence)
        return self.dropout(x)


学習用にマスク予測の層を追加する<br>
Next Sentence Prediction用のクラスは削除してある

In [8]:
class BERTLM(nn.Module):
    def __init__(self, bert: BERT, vocab_size):
        super().__init__()
        self.bert = bert
        self.mask_lm = MaskedLanguageModel(self.bert.hidden, vocab_size)

    def forward(self, x):
        x = self.bert(x)
        return self.mask_lm(x)

class MaskedLanguageModel(nn.Module):
    def __init__(self, hidden, vocab_size):
        super().__init__()
        self.linear = nn.Linear(hidden, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

In [9]:
class BERTHybrid(nn.Module):
    def __init__(self, bert: BERT, head):
        super().__init__()
        self.bert = bert
        self.epd = ErrorPositionDetectionHead(self.bert.hidden, head)
        self.esd = ErrorSentenceDetectionHead(head)
    def forward(self, x):
        x = self.bert(x)
        x = self.epd(x)
        return x, self.esd(x) 

class ErrorSentenceDetectionHead(nn.Module):
    def __init__(self, hidden):
        super().__init__()
        self.linear = nn.Linear(hidden, 2)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x[:, 0]))
    
class ErrorPositionDetectionHead(nn.Module):
    def __init__(self, hidden, head):
        super().__init__()
        self.linear = nn.Linear(hidden, head)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

BERT用のVocabを生成するクラスを定義する

In [10]:
import pickle
import tqdm
from collections import Counter

class TorchVocab(object):
    """
    :property freqs: collections.Counter, コーパス中の単語の出現頻度を保持するオブジェクト
    :property stoi: collections.defaultdict, string → id の対応を示す辞書
    :property itos: collections.defaultdict, id → string の対応を示す辞書
    """
    def __init__(self, counter, max_size=None, min_freq=1, specials=['<pad>', '<oov>'],
                 vectors=None, unk_init=None, vectors_cache=None):
        """
        :param coutenr: collections.Counter, データ中に含まれる単語の頻度を計測するためのcounter
        :param max_size: int, vocabularyの最大のサイズ. Noneの場合は最大値なし. defaultはNone
        :param min_freq: int, vocabulary中の単語の最低出現頻度. この数以下の出現回数の単語はvocabularyに加えられない.
        :param specials: list of str, vocabularyにあらかじめ登録するtoken
        :param vecors: list of vectors, 事前学習済みのベクトル. ex)Vocab.load_vectors
        """
        self.freqs = counter
        counter = counter.copy()
        min_freq = max(min_freq, 1)

        self.itos = list(specials)
        # special tokensの出現頻度はvocabulary作成の際にカウントされない
        for tok in specials:
            del counter[tok]

        max_size = None if max_size is None else max_size + len(self.itos)

        # まず頻度でソートし、次に文字順で並び替える
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)
        
        # 出現頻度がmin_freq未満のものはvocabに加えない
        for word, freq in words_and_frequencies:
            if freq < min_freq or len(self.itos) == max_size:
                break
            self.itos.append(word)

        # dictのk,vをいれかえてstoiを作成する
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}

        self.vectors = None
        if vectors is not None:
            self.load_vectors(vectors, unk_init=unk_init, cache=vectors_cache)
        else:
            assert unk_init is None and vectors_cache is None

    def __eq__(self, other):
        if self.freqs != other.freqs:
            return False
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        if self.vectors != other.vectors:
            return False
        return True

    def __len__(self):
        return len(self.itos)

    def vocab_rerank(self):
        self.stoi = {word: i for i, word in enumerate(self.itos)}

    def extend(self, v, sort=False):
        words = sorted(v.itos) if sort else v.itos
        for w in words:
            if w not in self.stoi:
                self.itos.append(w)
                self.stoi[w] = len(self.itos) - 1


class Vocab(TorchVocab):
    def __init__(self, counter, max_size=None, min_freq=1):
        self.pad_index = 0
        self.unk_index = 1
        self.eos_index = 2
        self.sos_index = 3
        self.mask_index = 4
        super().__init__(counter, specials=["<pad>", "<unk>", "<eos>", "<sos>", "<mask>"], max_size=max_size, min_freq=min_freq)

    # override用
    def to_seq(self, sentece, seq_len, with_eos=False, with_sos=False) -> list:
        pass

    # override用
    def from_seq(self, seq, join=False, with_pad=False):
        pass

    @staticmethod
    def load_vocab(vocab_path: str) -> 'Vocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            pickle.dump(self, f)


# テキストファイルからvocabを作成する
class WordVocab(Vocab):
    def __init__(self, texts, max_size=None, min_freq=1):
        print("Building Vocab")
        counter = Counter()
        for line in texts:
            if isinstance(line, list):
                words = line
            else:
                words = line.replace("\n", "").replace("\t", "").split()

            for word in words:
                counter[word] += 1
        super().__init__(counter, max_size=max_size, min_freq=min_freq)

    def to_seq(self, sentence, seq_len=None, with_eos=False, with_sos=False, with_len=False):
        if isinstance(sentence, str):
            sentence = sentence.split()

        seq = [self.stoi.get(word, self.unk_index) for word in sentence]

        if with_eos:
            seq += [self.eos_index]  # this would be index 1
        if with_sos:
            seq = [self.sos_index] + seq

        origin_seq_len = len(seq)

        if seq_len is None:
            pass
        elif len(seq) <= seq_len:
            seq += [self.pad_index for _ in range(seq_len - len(seq))]
        else:
            seq = seq[:seq_len]

        return (seq, origin_seq_len) if with_len else seq

    def from_seq(self, seq, join=False, with_pad=False):
        words = [self.itos[idx]
                 if idx < len(self.itos)
                 else "<%d>" % idx
                 for idx in seq
                 if not with_pad or idx != self.pad_index]

        return " ".join(words) if join else words

    @staticmethod
    def load_vocab(vocab_path: str) -> 'WordVocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)


def build(corpus_path, output_path, vocab_size=None, encoding='utf-8', min_freq=1):
    with open(corpus_path, "r", encoding=encoding) as f:
        vocab = WordVocab(f, max_size=vocab_size, min_freq=min_freq)

    print("VOCAB SIZE:", len(vocab))
    vocab.save_vocab(output_path)

Dataloaderを定義する.<br>
ここで文章中の単語をMASKする処理を行う<br>
Windowsでの並列化処理のために外部ファイル(dataset.py)で定義する  
MACやLinuxならこのipynbに直接定義しても良いはず

Trainerクラスを定義する.  
Masked Language Model : 文章中の一部の単語をマスクして,予測を行うタスク.  
- `save_bert`はbert本体のみを保存する
- `save_pretrain`は事前学習用のHeadも含んだ全体を保存する
- `load_bert`と`load_pretrain`も同様

In [11]:
import datetime
dt_now = str(datetime.datetime.now()).replace(' ', '')
dt_now = dt_now.replace(':','_')

In [12]:
from torch.utils.data import DataLoader
class HybridTrainer:
    def __init__(self, bert: BERT, train_dataloader: DataLoader, 
                 valid_dataloader: DataLoader = None,
                 lr: float = 1e-4, betas=(0.9, 0.999), weight_decay: float = 0.01,
                 with_cuda: bool = True, log_freq: int = 10, head = 9):
        # GPU環境において、GPUを指定しているかのフラグ
        cuda_condition = torch.cuda.is_available() and with_cuda
        self.device = torch.device("cuda:0" if cuda_condition else "cpu")

        self.bert = bert
        self.model = BERTHybrid(self.bert, head).to(self.device)

        if torch.cuda.device_count() > 1:
            print("Using %d GPUS for BERT" % torch.cuda.device_count())
            self.model = nn.DataParallel(self.model)

        self.train_data = train_dataloader
        self.valid_data = valid_dataloader

        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

        # masked_token予測のためのLoss関数を設定
        self.criterion = nn.NLLLoss()
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))
        
        self.train_losses = []
        self.valid_losses = []
        self.train_epd_accs = []
        self.valid_epd_accs = []
        self.train_esd_accs = []
        self.valid_esd_accs = []

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def valid(self, epoch):
        self.iteration(epoch, self.valid_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        str_code = "train" if train else "valid"
        data_iter = tqdm.tqdm(enumerate(data_loader), desc="EP_%s:%d" % (str_code, epoch), total=len(data_loader), bar_format="{l_bar}{r_bar}")
        avg_loss = 0.0
        total_epd_correct = 0
        total_esd_correct = 0
        total_element = 0
        for i, data in data_iter:
            data = {key: value.to(self.device) for key, value in data.items()}
            epd_output, esd_output = self.model.forward(data["bert_input"])
            epd_loss = self.criterion(epd_output.transpose(1, 2), data["token_label"])
            esd_loss = self.criterion(esd_output, data["label"])
            
            loss = epd_loss + esd_loss
            # training時のみ,backwardとoptimizer更新を行う
            if train:
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
            
            epd_correct = epd_output.argmax(dim=-1).eq(data["token_label"]).sum().item()
            esd_correct = esd_output.argmax(dim=-1).eq(data["label"]).sum().item()
            avg_loss += loss.item()
            total_epd_correct += epd_correct
            total_esd_correct += esd_correct
            total_element += data["label"].nelement()
            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_epd_acc": total_epd_correct / (total_element*128) * 100,
                "avg_esd_acc": total_esd_correct / total_element * 100,                
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))

        print("EP%d_%s, avg_loss=" % (epoch, str_code), avg_loss / len(data_iter),  
              "total_epd_acc=", total_epd_correct / (total_element*128) * 100,
                "total_esd_acc=", total_esd_correct * 100.0 / total_element)
        if train:
            self.train_losses.append(avg_loss / len(data_iter))
            self.train_epd_accs.append(total_epd_correct / (total_element*128) * 100 )
            self.train_esd_accs.append(total_esd_correct * 100.0 / total_element)
        else:
            self.valid_losses.append(avg_loss / len(data_iter))
            self.valid_epd_accs.append(total_epd_correct / (total_element*128) * 100)
            self.valid_esd_accs.append(total_esd_correct * 100.0 / total_element)
        
    def save_bert(self, epoch, file_path=output_dir + "bert_model"):
        output_path = file_path + ".ep%d" % epoch
        torch.save(self.bert.state_dict(), output_path)
        self.bert.to(self.device)
        print("EP:%d Model Saved on:" % epoch, output_path)
        return output_path
    
    def save_pretrain(self, epoch, file_path=output_dir + "mlm_model"):
        state = {
            'epoch' : epoch,
            'state_dict' : self.model.module.state_dict(),
            'optimizer' : self.optim.state_dict(),
            'train_epd_acc': self.train_epd_accs[-1],
            'train_esd_acc': self.train_esd_accs[-1],
            'valid_epd_acc': self.train_epd_accs[-1],
            'valid_esd_acc': self.train_esd_accs[-1]
        }
        output_path = file_path + ".ep%d" % epoch
        torch.save(state, output_path)
        self.bert.to(self.device)
        print("EP:%d Model Saved on:" % epoch, output_path)
        return output_path
 
    def load_bert(self, file_path):
        self.bert.state_dict(torch.load(file_path))
    
    def load_epd(self, file_path):
        state = torch.load(file_path)
        self.epd.load_state_dict(state['state_dict'])
    
    def load_pretrain(self, file_path):
        state = torch.load(file_path)
        #state = self.fix_model_state_dict(state)
        self.model.module.load_state_dict(state['state_dict'])
        self.optim.load_state_dict(state['optimizer'])
        return state

In [13]:
class MLMTrainer:
    def __init__(self, bert: BERT, vocab_size: int,
                 train_dataloader: DataLoader, valid_dataloader: DataLoader = None,
                 lr: float = 1e-4, betas=(0.9, 0.999), weight_decay: float = 0.01,
                 with_cuda: bool = True, log_freq: int = 10):
        # GPU環境において、GPUを指定しているかのフラグ
        cuda_condition = torch.cuda.is_available() and with_cuda
        self.device = torch.device("cuda:0" if cuda_condition else "cpu")
        self.bert = bert
        self.model = BERTLM(bert, vocab_size).to(self.device)

        if torch.cuda.device_count() > 1:
            print("Using %d GPUS for BERT" % torch.cuda.device_count())
            self.model = nn.DataParallel(self.model)

        self.train_data = train_dataloader
        self.valid_data = valid_dataloader

        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

        # masked_token予測のためのLoss関数を設定
        self.criterion = nn.NLLLoss()
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))
        
        self.train_losses = []
        self.valid_losses = []
        self.train_accs = []
        self.valid_accs = []

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def valid(self, epoch):
        self.iteration(epoch, self.valid_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        """
        :param epoch: 現在のepoch
        :param data_loader: torch.utils.data.DataLoader
        :param train: trainかtestかのbool値
        """
        str_code = "train" if train else "valid"
        data_iter = tqdm.tqdm(enumerate(data_loader), desc="EP_%s:%d" % (str_code, epoch), total=len(data_loader), bar_format="{l_bar}{r_bar}")
        avg_loss = 0.0
        total_correct = 0
        total_element = 0

        for i, data in data_iter:
            # 0. batch_dataはGPU or CPUに載せる
            data = {key: value.to(self.device) for key, value in data.items()}
            # 1. forward the next_sentence_prediction and masked_lm model
            mask_lm_output = self.model.forward(data["bert_input"])
            # 2-2. NLLLoss(negative log likelihood) : predicting masked token word
            mask_loss = self.criterion(mask_lm_output.transpose(1, 2), data["bert_label"])
            # 2-3. next_lossとmask_lossの合計をlossとする
            loss = mask_loss
            # 3. training時のみ,backwardとoptimizer更新を行う
            if train:
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()

            correct = mask_lm_output.argmax(dim=-1).eq(data["bert_label"]).sum().item()
            avg_loss += loss.item()
            total_element += data["bert_label"].nelement() * 128
            total_correct += correct
            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))
            
        if train:
            self.train_losses.append(avg_loss / len(data_iter))
            self.train_accs.append(total_correct * 100.0 / total_element)
        else:
            self.valid_losses.append(avg_loss / len(data_iter))
            self.valid_accs.append(total_correct * 100.0 / total_element)

        print("EP%d_%s, avg_loss=" % (epoch, str_code), avg_loss / len(data_iter))

        
    def save_bert(self, epoch, file_path=output_dir + "bert"):
        output_path = file_path + ".ep%d" % epoch
        torch.save(self.bert.state_dict(), output_path)
        self.bert.to(self.device)
        print("EP:%d Model Saved on:" % epoch, output_path)
        return output_path
    
    def save_pretrain(self, epoch, file_path=output_dir + "mlm"):
        state = {
            'epoch' : epoch,
            'state_dict' : self.model.module.state_dict(),
            'optimizer' : self.optim.state_dict(),
            'train_loss' : self.train_losses,
            'valid_loss' : self.valid_losses
        }
        output_path = file_path + ".ep%d" % epoch
        torch.save(state, output_path)
        self.bert.to(self.device)
        print("EP:%d Model Saved on:" % epoch, output_path)
        return output_path
 
    def load_bert(self, file_path):
        self.bert.state_dict(torch.load(file_path))
        
    def load_pretrain(self, file_path):
        state = torch.load(file_path)
        self.model.module.load_state_dict(state['state_dict'])
        self.optim.load_state_dict(state['optimizer'])
        self.train_losses = state['train_loss']
        self.valid_losses = state['valid_loss']
        return state

In [14]:
import matplotlib.pyplot as plt

import matplotlib.ticker as ticker
import numpy as np
plt.style.use('ggplot')

def showPlot(points):

    fig, ax = plt.subplots()
    plt.plot(points)
    plt.show()
    
def savePlot(points, figure_path):
    plt.switch_backend('Agg')
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig(figure_path)

In [15]:
# 訓練用パラメタを定義する
vocab_path= read_dir + 'vocab.txt'

hidden=256 #768
layers=2 #12
attn_heads=4 #12
seq_len=128

batch_size=128
num_workers=0
with_cuda=True
log_freq=100
corpus_lines=None

lr=1e-3
adam_weight_decay=0.00
adam_beta1=0.9
adam_beta2=0.999

dropout=0.0

min_freq=7

label_path=None

In [16]:
#corpus_path=processed_train_txt
#build(corpus_path, vocab_path, min_freq=min_freq)

In [17]:

print("Loading Vocab", vocab_path)
vocab = WordVocab.load_vocab(vocab_path)

from dataset import MixDataset
print("Loading Train Dataset", processed_train_txt)
train_dataset = MixDataset(processed_train_txt, vocab, seq_len=seq_len, label_path=label_path, corpus_lines=corpus_lines)

print("Loading Valid Dataset", processed_valid_txt)
valid_dataset = MixDataset(processed_valid_txt, vocab, seq_len=seq_len, label_path=label_path) if processed_valid_txt is not None else None

print("Creating Dataloader")
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers) if processed_valid_txt is not None else None

Loading Vocab ./data/merged_bert/vocab.txt
Loading Train Dataset ./data/merged_bert/train_X.txt
Loading Valid Dataset ./data/merged_bert/valid_X.txt
Creating Dataloader


In [18]:
print("Building BERT model")
hybrid_bert = BERT(len(vocab), hidden=hidden, n_layers=layers, attn_heads=attn_heads, dropout=dropout)
hybrid_trainer = HybridTrainer(hybrid_bert, train_dataloader=train_data_loader, valid_dataloader=valid_data_loader,lr=lr, 
                            betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay,with_cuda=with_cuda, 
                            log_freq=log_freq, head=9)


layers=4 #12
attn_heads=8 #12
mlm_bert = BERT(len(vocab), hidden=hidden, n_layers=layers, attn_heads=attn_heads, dropout=dropout)
mlm_trainer = MLMTrainer(mlm_bert, len(vocab), train_dataloader=train_data_loader, valid_dataloader=valid_data_loader,
                  lr=lr, betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay,
                  with_cuda=with_cuda, log_freq=log_freq)


Building BERT model
Using 2 GPUS for BERT
Total Parameters: 2761501
Using 2 GPUS for BERT
Total Parameters: 5522944


In [19]:
hybrid_model_path = output_dir + 'mix/hybrid/small.ep26'
mlm_model_path = output_dir + 'mlm/mid/.ep16'
_ = hybrid_trainer.load_pretrain(hybrid_model_path)

_ = mlm_trainer.load_pretrain(mlm_model_path)


In [22]:
test_dataset = MixDataset(processed_test_txt, vocab, seq_len=seq_len, label_path=label_path, is_train=False)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=num_workers, shuffle=True)

In [44]:
def test_all():
    hybrid_trainer.model.eval()
    mlm_trainer.model.eval()
    with torch.no_grad():
        for batch in test_loader:
            input_line = []
            text = batch['bert_input'][0]
            original = batch['original'][0]
            epd_output, esd_output = hybrid_trainer.model(batch['bert_input'])
            org_text = [vocab.itos[i] for i in original if i!=0]
            print("".join(org_text))
            processed_text = [vocab.itos[i] for i in text if i!=0]
            processed_text = processed_text[1:-1]
            print("".join(processed_text))
            print(batch['label'].item(), esd_output.argmax(dim=-1).item())
            
            #NG文だけ次のステップへ
            if esd_output.argmax(dim=-1).item()==1:
                predict_list = []
                masked_list = []
                masked_label = []
                for j in range(len(text)):
                    predict = epd_output[0,j].argmax(dim=-1).item()
                    if predict == 0:
                        # 正しい場合
                        masked_list.append(text[j])
                        masked_label.append(-1)
                        predict_list.append(0)
                    elif predict == 1:
                        # 脱字(1文字)
                        masked_list.extend([text[j], vocab.mask_index])
                        masked_label.extend([-1,-2])
                        predict_list.extend([1,-1])
                    elif predict == 2:
                        # 脱字(2文字)
                        masked_list.extend([text[j],vocab.mask_index, vocab.mask_index])
                        masked_label.extend([-1,-2,-2])
                        predict_list.extend([2,-1,-1])
                    elif predict == 3:
                        # 誤字
                        masked_list.append(vocab.mask_index)
                        masked_label.append(text[j])
                        predict_list.append(3)
                    elif predict == 4:
                        # 誤字かつ脱字(1文字)
                        masked_list.extend([vocab.mask_index, vocab.mask_index])
                        masked_label.extend([text[j],-2])
                        predict_list.extend([4,-1])
                    elif predict == 5:
                        # 誤字かつ脱字(2文字)
                        masked_list.extend([vocab.mask_index, vocab.mask_index, vocab.mask_index])
                        masked_label.extend([text[j],-2,-2])
                        predict_list.extend([5,-1,-1])
                    elif predict == 6:
                        # 衍字
                        predict_list.append(6)
                    elif predict == 7:
                        # 衍字かつ脱字(1文字)
                        masked_list.append(vocab.mask_index)
                        masked_label.append(-2)
                        predict_list.extend([7,-1])
                    elif predict == 8:
                        # 衍字かつ脱字(2文字)
                        masked_list.extend([vocab.mask_index, vocab.mask_index])
                        masked_label.extend(-2,-2)
                        predict_list.extend([8,-1,-1])
                no_pad_text = [vocab.itos[i] for i in masked_list if i!=0]
                no_pad_text = no_pad_text[1:-1]
                print("".join(no_pad_text))
                masked_list = masked_list[:128]
                masked_label = masked_label[:128]
                predict_list = predict_list[:128]
                masked_text = [vocab.itos[i] for i in masked_list]
                masked_batch = torch.tensor(masked_list)
                masked_batch = masked_batch.unsqueeze(0)
                mlm_output = mlm_trainer.model(masked_batch)
                print(len(epd_output[0]))
                print(len(masked_text))
                enji_flag = False
                for j in range(len(masked_text)):
                    _, topi = mlm_output[0, j].topk(3)
                    predict = [vocab.itos[index.item()] for index in topi]
                    if masked_list[j] in [0,2,3]:
                        continue

                    if predict_list[j] in [6,7,8]:
                        print(predict_list[j],":<del>")

                    print(predict_list[j],":",end="")
                    if masked_label[j] == -1:
                        print(masked_text[j])
                    elif masked_label[j] == -2:
                        print(masked_text[j],':脱:',predict)
                    else:
                        print(masked_text[j],':誤:',predict)

test_all()

議論がコソボに集中すれば、従来にも増してサミットでの日本の存在感がかすむのは確実。
議論がコソボに集中すすば、従来には増してサミットでの日本の存在感がかすむは確実。
1 1
議論がコソボに集中すば、従来には増してサミットでの日本の存在感がかす<mask>確実。
128
126
0 :議
0 :論
0 :が
0 :コ
0 :ソ
0 :ボ
0 :に
0 :集
0 :中
6 :<del>
6 :す
0 :ば
0 :、
0 :従
0 :来
0 :に
0 :は
0 :増
0 :し
0 :て
0 :サ
0 :ミ
0 :ッ
0 :ト
0 :で
0 :の
0 :日
0 :本
0 :の
0 :存
0 :在
0 :感
0 :が
0 :か
0 :す
0 :<mask> :誤: ['が', 'も', 'は']
3 :確
6 :<del>
6 :実
0 :。
バブル崩壊後の株式市況低迷で業績が悪化していたのに加え、１９９９年の株式委託手数料完全自由化をにらみ、単独での生き残りは難しいと判断した。外資系証券との競争が本格化する中、日興も系列証券を支援する余力は限られており、合併によるリストラで、系列証券の体質強化を図る。
バブル崩壊後のの株式市況低迷でで業績が悪化ていたのに加え、１９９９年の株式委託手数料完全自由化をにみ、単独の生き残りも難しいと判断した。外資系証券と競争が本格化する中、日興も系列証券を支援る余力は限らｒており、合併よるリストラで、系列証券の体質強化を図
1 1
バブル崩壊後のの株式市況低迷でで業績が悪化ていたのに加え、１９９９年の株式委託手数料完全自由化をにみ、単独の生き残りも難しいと判断した。外資系証券と競争が本格化する中、日興も系列証券を支援る余力は限ら<mask>ており、合併よるリストラで、系列証券の体質強化を
128
127
0 :バ
0 :ブ
0 :ル
0 :崩
0 :壊
0 :後
0 :の
0 :の
0 :株
0 :式
0 :市
0 :況
0 :低
0 :迷
0 :で
0 :で
0 :業
0 :績
0 :が
0 :悪
0 :化
0 :て
0 :い
0 :た
0 :の
0 :に
0 :加
0 :え
0 :、
0 :１
0 :９
0 :９
0 :９
0 :年
0 :の
0 :株
0 :式
0 :委
0 :託
0 :手
0 :数
0 :料
0 :完
0 :全
0 :自


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



:工
0 :乳
0 :と
0 :乳
0 :飲
0 :料
0 :の
0 :商
0 :品
0 :名
0 :に
0 :「
0 :牛
0 :乳
0 :」
0 :文
0 :言
3 :<mask> :誤: ['を', 'で', 'が']
0 :使
0 :用
0 :す
0 :る
0 :場
0 :合
0 :、
0 :従
0 :来
0 :の
0 :成
0 :分
0 :規
0 :定
0 :に
0 :加
0 :え
6 :<del>
6 :、
0 :新
0 :た
0 :に
0 :「
0 :生
0 :乳
0 :を
0 :５
0 :０
0 :％
0 :超
0 :使
0 :用
0 :<mask> :誤: ['す', 'せ', 'あ']
3 :る
0 :こ
0 :と
0 :」
0 :を
0 :要
0 :件
0 :に
0 :加
0 :た
6 :<del>
6 :。
その後、男性が持つＹ染色体を調べた研究でも現代男性の祖先は１９万年前のアフリカ男性にたどり着いている。
そ後、男性がが持つＹ染色体を調にた研究も現代男性の祖先は１９万年前のアフリカ男性にたどさ着いといる。
1 1
そ後、男性が持つＹ染色体を調にた研究も現代男性の祖先は１９万年前のアフリカ男性にた<mask>さ<mask>着いといる。
128
128
0 :そ
0 :後
0 :、
0 :男
0 :性
0 :が
6 :<del>
6 :持
0 :つ
0 :Ｙ
0 :染
0 :色
0 :体
0 :を
0 :調
0 :に
0 :た
0 :研
0 :究
0 :も
0 :現
0 :代
0 :男
0 :性
0 :の
0 :祖
0 :先
0 :は
0 :１
0 :９
0 :万
0 :年
0 :前
0 :の
0 :ア
0 :フ
0 :リ
0 :カ
0 :男
0 :性
0 :に
0 :た
0 :<mask> :誤: ['く', 'た', 'る']
3 :さ
1 :<mask> :脱: ['れ', 'え', 'が']
-1 :着
0 :い
0 :と
0 :い
0 :る
0 :。
３年前、自民党に推されて北海道知事選に出馬、敗れて、いまは弁護士稼業である。
３年前、自民党に推ｓれて北海道知事選出馬、敗れて、まは弁護士稼業である。
1 1
３年前、自民党に推<mask>れて北海道知事選出馬、敗れて、まは弁護士稼業である。
128
128
0 :３
0 :年
0 :

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 :面
0 :で
0 :な
0 :く
0 :百
0 :面
0 :相
0 :で
0 :応
0 :る
0 :時
0 :、
0 :べ
0 :の
0 :人
0 :は
0 :快
0 :調
0 :と
0 :み
0 :て
0 :い
0 :い
0 :。
中小零細企業や個人に適用される現行の和議法に代わる法律として政府が制定を予定しており、成果が不確実な和議手続きの欠陥を補いながらスピーディーな簡易型手続きを盛り込んだのが特徴だ。
中小零細企業ｙ個人に適用される現行の和議法に代わる法律とでて政府がが制定を予定ししておおり、成果が不確実な和議手続の欠陥を補いがらスピーディー簡易型手続きを盛り込んだだののが特徴だ。
1 1
中小零細企業<mask>個人に適用される現行の和議法に代わる法律とでて政府がが制定を予定ししており、成果が不確実な和議手続の欠陥を補いがらスピーディー簡易型手続きを盛り込んだののが特徴。
128
125
0 :中
0 :小
0 :零
0 :細
0 :企
0 :業
3 :<mask> :誤: ['や', 'が', 'は']
0 :個
0 :人
0 :に
0 :適
0 :用
0 :さ
0 :れ
0 :る
0 :現
0 :行
0 :の
0 :和
0 :議
0 :法
0 :に
0 :代
0 :わ
0 :る
0 :法
0 :律
0 :と
0 :で
0 :て
0 :政
0 :府
0 :が
0 :が
0 :制
0 :定
0 :を
0 :予
0 :定
0 :し
0 :し
0 :て
6 :<del>
6 :お
0 :り
0 :、
0 :成
0 :果
0 :が
0 :不
0 :確
0 :実
0 :な
0 :和
0 :議
0 :手
0 :続
0 :の
0 :欠
0 :陥
0 :を
0 :補
0 :い
0 :が
0 :ら
0 :ス
0 :ピ
0 :ー
0 :デ
0 :ィ
0 :ー
0 :簡
0 :易
0 :型
0 :手
0 :続
0 :き
0 :を
0 :盛
0 :り
0 :込
0 :ん
0 :だ
0 :の
6 :<del>
6 :の
0 :が
0 :特
0 :徴
0 :。
近く自由党も交えて最終的な合意書を取り交わす。
近自由党交えて最終的ｎ合意書を取り交す。
1 1
近自由党交えて最終的<mask>合意書を取り交す。
128
128
0 :近
0 :自
0 :由
0 :党
0 :交
0 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0 :先
0 :］
0 :４
0 :三
0 :角
0 :成
0 :［
0 :後
0 :］
0 :同
0 :金
0 :［
0 :先
0 :］
0 :５
0 :二
0 :銀
0 :に
0 :い
0 :い
0 :。
「介助犬シンシアのまち」を宣言するなどしてきた宝塚から、「だれにもやさしい街」づくりのメッセージを発信。
「介助犬シンシアのまちち」を宣言するなしてき宝塚か、「だにもしい街」づづくりのメッセージを発信。
1 1
「介助犬シンシアのまち」を宣言するなしてき宝塚か、「だにもしい街」づくりのメッセージを発信。
128
126
0 :「
0 :介
0 :助
0 :犬
0 :シ
0 :ン
0 :シ
0 :ア
0 :の
0 :ま
0 :ち
6 :<del>
6 :」
0 :を
0 :宣
0 :言
0 :す
0 :る
0 :な
0 :し
0 :て
0 :き
0 :宝
0 :塚
0 :か
0 :、
0 :「
0 :だ
0 :に
0 :も
0 :し
0 :い
0 :街
0 :」
0 :づ
0 :く
6 :<del>
6 :り
0 :の
0 :メ
0 :ッ
0 :セ
0 :ー
0 :ジ
0 :を
0 :発
0 :信
0 :。
このおかげで、中小保険会社は手厚い保護を受けてきた。
こｎかげで、中小保険会社ｈ手厚い保護を受けてきた。
1 1
こ<mask>か<mask>で、中小保険会社<mask>手厚い保護を受けてきた。
128
128
0 :こ
3 :<mask> :誤: ['れ', 'こ', 'の']
0 :か
3 :<mask> :誤: ['ら', 'り', 'け']
0 :で
0 :、
0 :中
0 :小
0 :保
0 :険
0 :会
0 :社
3 :<mask> :誤: ['が', 'は', 'の']
0 :手
0 :厚
0 :い
0 :保
0 :護
0 :を
0 :受
0 :け
0 :て
0 :き
0 :た
0 :。
勝負の行方は、加藤が小駒だけで、果たして先手玉を捕らえ切れるかどうかにかかってきたが、先に１分将棋に入った田中に、ついに最後の最後の敗着が出た。
勝負の行方は、加藤ど小駒だけで、果たして先手玉を捕らえ切れわかどかかかっばきた、先に１分将棋入ったた田中に、いに最後の最後の敗着ど出た。
1 1
勝負の行方は、加藤ど小駒だけで、果たして先手玉を捕らえ切れわかどかか

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 :い
0 :戦
0 :い
0 :続
6 :<del>
6 :く
0 :Ｗ
0 :杯
0 :で
0 :壁
0 :に
0 :当
0 :た
0 :る
0 :の
0 :か
0 :、
6 :<del>
6 :一
0 :段
0 :と
0 :飛
0 :躍
0 :す
0 :る
0 :の
0 :の
0 :か
0 :。
0 :目
0 :が
0 :が
0 :離
0 :せ
0 :<mask> :誤: ['な', 'た', 'ま']
0 :い
3 :。
その前にケリをつけておきたいことがある。
その前にケリをつけておきたいことがある。
0 0
また、中国市場をめぐる日米間の攻防や米国の対日姿勢にも微妙な変化をもたらしそうだ。
また、中国市場をめぐる日米間の攻防や米国のの対日姿勢ににも微妙変化がもたらししそだ。
1 1
また、中国市場をめぐる日米間の攻防や米国のの対日姿勢ににも微妙変化がもたらしそだ。
128
127
0 :ま
0 :た
0 :、
0 :中
0 :国
0 :市
0 :場
0 :を
0 :め
0 :ぐ
0 :る
0 :日
0 :米
0 :間
0 :の
0 :攻
0 :防
0 :や
0 :米
0 :国
0 :の
0 :の
0 :対
0 :日
0 :姿
0 :勢
0 :に
0 :に
0 :も
0 :微
0 :妙
0 :変
0 :化
0 :が
0 :も
0 :た
0 :ら
0 :し
6 :<del>
6 :そ
0 :だ
0 :。
残念ながら私は読んでいないのだが、「コレデオシマイ」といういわば後編があって、こちらは二匹目のどじょうのようなのだが、これだけ読んでも十分に楽しめる。
残念なががら私は読んでいｎいのだが、「コレデオシマイ」といいういわば後編あって、ここちらは二匹目のどじょうののようっのだが、こｒだけ読んでも十分に楽しめｒ。
1 1
残念なががら私は読んでい<mask>いのだが、「コレデオシマイ」といいういわば後編あって、こちらは二匹目のどじょうののようっのだが、こ<mask>け読んでも十分に楽しめ<mask>。
128
126
0 :残
0 :念
0 :な
0 :が
0 :が
0 :ら
0 :私
0 :は
0 :読
0 :ん
0 :で
0 :い
3 :<mask> :誤: ['な', 'た', 'る']
0 :い
0 :の
0 :だ
0 :が
0 :、
0 :「
0 :コ


KeyboardInterrupt: 

In [80]:
def test_input(original_text ,processed_text):
    hybrid_trainer.model.eval()
    mlm_trainer.model.eval()
    with torch.no_grad():
        input_line = []
        original = original_text
        processed_index = [vocab.sos_index] +[vocab.stoi[i] for i in processed_text] + [vocab.eos_index]
        text = torch.tensor(processed_index)
        processed_batch = text.unsqueeze(0)
        epd_output,esd_output = hybrid_trainer.model(processed_batch)
        print(original_text)
        print(processed_text)
        #NG文だけ次のステップへ
        print(esd_output.argmax(dim=-1).item())
        #if esd_output.argmax(dim=-1).item()==1:
        predict_list = []
        masked_list = []
        masked_label = []
        for j in range(len(text)):
            predict = epd_output[0,j].argmax(dim=-1).item()
            if predict == 0:
                # 正しい場合
                masked_list.append(text[j])
                masked_label.append(-1)
                predict_list.append(0)
            elif predict == 1:
                # 脱字(1文字)
                masked_list.extend([text[j], vocab.mask_index])
                masked_label.extend([-1,-2])
                predict_list.extend([1,-1])
            elif predict == 2:
                # 脱字(2文字)
                masked_list.extend([text[j],vocab.mask_index, vocab.mask_index])
                masked_label.extend([-1,-2,-2])
                predict_list.extend([2,-1,-1])
            elif predict == 3:
                # 誤字
                masked_list.append(vocab.mask_index)
                masked_label.append(text[j])
                predict_list.append(3)
            elif predict == 4:
                # 誤字かつ脱字(1文字)
                masked_list.extend([vocab.mask_index, vocab.mask_index])
                masked_label.extend([text[j],-2])
                predict_list.extend([4,-1])
            elif predict == 5:
                # 誤字かつ脱字(2文字)
                masked_list.extend([vocab.mask_index, vocab.mask_index, vocab.mask_index])
                masked_label.extend([text[j],-2,-2])
                predict_list.extend([5,-1,-1])
            elif predict == 6:
                # 衍字
                predict_list.append(6)
            elif predict == 7:
                # 衍字かつ脱字(1文字)
                masked_list.append(vocab.mask_index)
                masked_label.append(-2)
                predict_list.extend([7,-1])
            elif predict == 8:
                # 衍字かつ脱字(2文字)
                masked_list.extend([vocab.mask_index, vocab.mask_index])
                masked_label.extend(-2,-2)
                predict_list.extend([8,-1,-1])
        no_pad_text = [vocab.itos[i] for i in masked_list if i!=0]
        no_pad_text = no_pad_text[1:-1]
        print("".join(no_pad_text))
        masked_list = masked_list[:128]
        masked_label = masked_label[:128]
        predict_list = predict_list[:128]
        masked_text = [vocab.itos[i] for i in masked_list]
        masked_batch = torch.tensor(masked_list)
        masked_batch = masked_batch.unsqueeze(0)
        mlm_output = mlm_trainer.model(masked_batch)
        print(len(epd_output[0]))
        print(len(masked_text))
        enji_flag = False
        for j in range(len(masked_text)):
            _, topi = mlm_output[0, j].topk(3)
            predict = [vocab.itos[index.item()] for index in topi]
            if masked_list[j] in [0,2,3]:
                continue

            if predict_list[j] in [6,7,8]:
                print(predict_list[j],":<del>")

            print(predict_list[j],":",end="")
            if masked_label[j] == -1:
                print(masked_text[j])
            elif masked_label[j] == -2:
                print(masked_text[j],':脱:',predict)
            else:
                print(masked_text[j],':誤:',predict)
test_input("政党が合意に達しｔことを評価している","奨学金は、経済的理由に就学困難な者に与えるものだ。")

政党が合意に達しｔことを評価している
奨学金は、経済的理由に就学困難な者に与えるものだ。
0
奨学金は、経済的理由に就学困難な者に与えるものだ。
27
27
0 :奨
0 :学
0 :金
0 :は
0 :、
0 :経
0 :済
0 :的
0 :理
0 :由
0 :に
0 :就
0 :学
0 :困
0 :難
0 :な
0 :者
0 :に
0 :与
0 :え
0 :る
0 :も
0 :の
0 :だ
0 :。


In [ ]:
def test_esd():
    hybrid_trainer.model.eval()
    with torch.no_grad():
        n_correct = 0
        true_positive = 0
        true_negative = 0
        false_positive = 0
        false_negative = 0
        for i, batch in enumerate(tqdm.tqdm(test_loader)):
            if i==1000:
                break
            input_line = []
            _, esd_output = hybrid_trainer.model(batch['bert_input'])
            if batch['label'][0] == 1:
                if esd_output.argmax(dim=-1).item() == 1:
                    #NG文をNGと判定したとき
                    true_positive += 1
                    n_correct += 1
                else:
                    #NG文をOKと判定したとき
                    false_negative += 1
            else:
                if esd_output.argmax(dim=-1).item() == 1:
                    #OK文をNGと判定したとき
                    false_positive += 1
                else:
                    #OK文をOKと判定したとき
                    n_correct += 1
                    true_negative += 1
            print("検出率："+ str(n_correct/(i+1)))
        print("再現率：", true_positive/(true_positive+false_negative))
        print("適合率：", true_positive/(true_positive+false_positive))
        print("特異度：", true_negative/(false_positive+true_negative))
        print("F値：", 2*true_positive/(2*true_positive+false_negative+false_positive))

    with open("./hybrid_single/esd.txt","w", encoding='utf-8') as f:
        f.write("検出率："+ str(n_correct/i)+"\n")
        f.write("再現率："+ str(true_positive/(true_positive+false_negative))+"\n")
        f.write("適合率："+ str(true_positive/(true_positive+false_positive))+"\n")
        f.write("特異度："+ str(true_negative/(false_positive+true_negative))+"\n")
        f.write("F値："+ str(2*true_positive/(2*true_positive+false_negative+false_positive))+"\n")

test_esd()

In [ ]:
def test_mlm():
    mlm_trainer.model.eval()
    with torch.no_grad():
        total_correct = 0
        total_elements = 0
        for  i, batch in enumerate(tqdm.tqdm(test_loader)):
            if i==1000:
                break
            input_line = []
            output = mlm_trainer.model(batch['bert_input'])[0]
            masked_label = batch["token_label"][0]
            correct = 0
            elements = 0
            for j in range(len(batch['bert_input'][0])):
                if masked_label[j] != 0:
                    elements += 1
                    _, topi = output[j].topk(3)
                    if masked_label[j].item() in topi:
                        correct += 1
            total_correct += correct
            total_elements += elements
            print("total_acc;", total_correct / total_elements *100)
            
test_mlm()

In [ ]:
def test_epd():
    hybrid_trainer.model.eval()
    with torch.no_grad():
        all_num = 0
        n_correct = 0
        p_correct = 0
        true_positive = 0
        true_negative = 0
        false_positive = 0
        false_negative = 0
        for i, batch in enumerate(tqdm.tqdm(test_loader)):
            if i==1000:
                break
            output,_ = hybrid_trainer.model(batch['bert_input'])
            for j in range(128):
                all_num += 1
                true =  batch["token_label"][0,j].item()
                predict = output[0,j].argmax(dim=-1).item()
                if true != 0:
                    if predict != 0:
                        true_positive += 1
                        n_correct += 1
                        if predict == true:
                            p_correct += 1
                    else:
                        false_negative += 1
                else:
                    if predict != 0:
                        false_positive += 1
                    else:
                        true_negative += 1
                        n_correct += 1
            print("検出率：", n_correct/all_num)
            #print("完全検出率：", p_correct/all_num)
        print("再現率：", true_positive/(true_positive+false_negative))
        print("適合率：", true_positive/(true_positive+false_positive))
        print("特異度：", true_negative/(false_positive+true_negative))
        print("F値：", 2*true_positive/(2*true_positive+false_negative+false_positive))
    
    with open("./hybrid_single/epd.txt","w", encoding='utf-8') as f:
        f.write("検出率："+ str(n_correct/all_num)+"\n")
        f.write("再現率："+ str(true_positive/(true_positive+false_negative))+"\n")
        f.write("適合率："+ str(true_positive/(true_positive+false_positive))+"\n")
        f.write("特異度："+ str(true_negative/(false_positive+true_negative))+"\n")
        f.write("F値："+ str(2*true_positive/(2*true_positive+false_negative+false_positive))+"\n")
test_epd()

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
import matplotlib.ticker as ticker
import numpy as np
plt.style.use('ggplot')

def showPlot(points, figure_path):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig(figure_path)

In [ ]:
showPlot(trainer.train_losses, "./results/"+str(epochs)+"_train.png")
showPlot(trainer.valid_losses, "./results/"+str(epochs)+"_valid.png")